<a href="https://colab.research.google.com/github/wangbxj1234/xformerforpt/blob/main/cosformer%E4%BB%A3%E7%A0%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

from torch import Tensor
from typing import Optional
from torch import nn
class CosformerAttention1(nn.Module):
    """
    cosformer attention in "cosFormer: Rethinking Softmax In Attention"
    https://arxiv.org/abs/2202.08791
    """
    def __init__(self,in_planes):
        super().__init__()

        self.act_fun = self.get_act_fun()
        # q, k, v projection
        self.k_proj = nn.Linear(in_planes, in_planes)
        self.v_proj = nn.Linear(in_planes, in_planes)
        self.q_proj = nn.Linear(in_planes, in_planes)
        #self.p_proj = nn.Linear(3, in_planes)
        self.p_proj = nn.Sequential(nn.Linear(3, 3), nn.BatchNorm1d(3), nn.ReLU(inplace=True), nn.Linear(3, in_planes))
        # outprojection
        self.out_proj = nn.Linear(in_planes, in_planes)

    def get_index(self, seq_len):
        index = np.pi / 2 * torch.arange(1, seq_len + 1).reshape(1, -1, 1)
        return nn.Parameter(index, requires_grad=False)

    def get_act_fun(self):
        return F.relu
    def forward(self, pxo) -> torch.Tensor:
        p, x, o = pxo 
        eps = 1e-6
     
        n=0;
        x_sub=[]
        p_sub=[]
        out=[]
        for i in o:
           x_s=x[n:int(i),:]
           x_sub.append(x_s)
           p_s=p[n:int(i),:]
           p_sub.append(p_s)
           tgt_len, embed_dim = x_s.size() ###(N,d) 

           n=int(i)

           q = self.q_proj(x_s)######
        # (N,d)
           k = self.k_proj(x_s)########
        # (N,d)
           v = self.v_proj(x_s)######
           
           #pe = self.p_proj(p_s)

    
           q = q#+pe
           k = k#+pe
        # activation
           q = self.act_fun(q)####Q'
           k = self.act_fun(k)####K'
        
        # cos transform
           m = tgt_len
        # get index and send to cuda
           weight_index = self.get_index(m).to(q)########(pi*[1,m,1])/2

        # (N, 2 * d)
           q_ = torch.cat([q * torch.sin(weight_index[:tgt_len, :] / m), q * torch.cos(weight_index[ :tgt_len, :] / m)], dim=-1).squeeze(0)
        # (N, 2 * d)
           k_ = torch.cat([k * torch.sin(weight_index[:tgt_len, :] / m), k * torch.cos(weight_index[ :tgt_len, :] / m)], dim=-1).squeeze(0)

        
            # ( L, 2 * d) ( L, d) -> ( 2 * d, d)
           kv_ = torch.einsum('ld,lm->dm', k_, v)
            # ( L, 2 * d) ( 2 * d) -> ( L)
           z_ = 1 / torch.clamp_min(torch.einsum('ld,d->l', q_, torch.sum(k_, axis=0)), eps)
            # ( L, 2 * d) ( d, 2 * d) ( L) -> ( L, d)
           attn_output = self.out_proj(torch.einsum('ld,dm,l->lm', q_, kv_, z_))
           out.append(attn_output)
        #out=torch.IntTensor(out)
        out=torch.cat(out,dim=0)

        return out
class CosformerAttention(nn.Module):
    """
    cosformer attention in "cosFormer: Rethinking Softmax In Attention"
    https://arxiv.org/abs/2202.08791
    """
    def __init__(self,in_planes):
        super().__init__()

        self.act_fun = self.get_act_fun()
        # q, k, v projection
        self.k_proj = nn.Linear(in_planes, in_planes)
        self.v_proj = nn.Linear(in_planes, in_planes)
        self.q_proj = nn.Linear(in_planes, in_planes)
        self.p_proj = nn.Linear(3, in_planes)
        # outprojection
        self.out_proj = nn.Linear(in_planes, in_planes)

    def get_index(self, seq_len):
        index = np.pi / 2 * torch.arange(1, seq_len + 1).reshape(1, -1, 1)
        return nn.Parameter(index, requires_grad=False)

    def get_act_fun(self):
        return F.relu
    def forward(self, pxo) -> torch.Tensor:
        p, x, o = pxo 
        tgt_len, embed_dim = x.size() ###(N,d) 
        eps = 1e-6
        # get q, k, v
        # (N,d)       
        #q = self.q_proj(x)######
        q_p=nn.Linear(128, 128)
        q=q_p(x)
        # (N,d)
        k = self.k_proj(x)########
        # (N,d)
        v = self.v_proj(x)######
        pe= self.p_proj(p)
        q = q+pe
        k = k+pe
        # activation
        q = self.act_fun(q)####Q'
        k = self.act_fun(k)####K'
        
        # cos transform
        m = tgt_len
        # get index and send to cuda
        weight_index = self.get_index(m).to(q)########(pi*[1,m,1])/2

        # (N, 2 * d)
        q_ = torch.cat([q * torch.sin(weight_index[:tgt_len, :] / m), q * torch.cos(weight_index[ :tgt_len, :] / m)], dim=-1).squeeze(0)
        # (N, 2 * d)
        k_ = torch.cat([k * torch.sin(weight_index[:tgt_len, :] / m), k * torch.cos(weight_index[ :tgt_len, :] / m)], dim=-1).squeeze(0)

        
            # ( L, 2 * d) ( L, d) -> ( 2 * d, d)
        kv_ = torch.einsum('ld,lm->dm', k_, v)
            # ( L, 2 * d) ( 2 * d) -> ( L)
        z_ = 1 / torch.clamp_min(torch.einsum('ld,d->l', q_, torch.sum(k_, axis=0)), eps)
            # ( L, 2 * d) ( d, 2 * d) ( L) -> ( L, d)
        attn_output = torch.einsum('ld,dm,l->lm', q_, kv_, z_)

        x = self.out_proj(attn_output)

        return x

In [ ]:
o=[ 838, 2088, 2317, 3567, 4541, 5561, 6427, 7077]
o=torch.IntTensor(o)
x = torch.rand(7077, 128)
p = torch.rand(7077, 3)

In [ ]:
planes=128
transformer2 = CosformerAttention1(planes)
right_res = transformer2([p, x, o])
print(right_res.shape)

torch.Size([7077, 128])


In [ ]:
class CosformerAttention1(nn.Module):
    """
    cosformer attention in "cosFormer: Rethinking Softmax In Attention"
    https://arxiv.org/abs/2202.08791
    """
    def __init__(self,in_planes):
        super().__init__()

        self.act_fun = self.get_act_fun()
        # q, k, v projection
        self.k_proj = nn.Linear(in_planes, in_planes)
        self.v_proj = nn.Linear(in_planes, in_planes)
        self.q_proj = nn.Linear(in_planes, in_planes)
        self.p_proj = nn.Linear(3, in_planes)
        self.linear_p = nn.Sequential(nn.Linear(3, 3), nn.BatchNorm1d(3), nn.ReLU(inplace=True), nn.Linear(3, in_planes))
        # outprojection
        self.out_proj = nn.Linear(in_planes, in_planes)
   
    def get_index(self, seq_len):
        index = np.pi / 2 * torch.arange(1, seq_len + 1).reshape(1, -1, 1)
        return nn.Parameter(index, requires_grad=False)

    def get_act_fun(self):
        return F.relu
    def forward(self, pxo) -> torch.Tensor:
        p, x, o = pxo 
        eps = 1e-6
     
        n=0;
        x_sub=[]
        p_sub=[]
        out=[]
        for i in o:
           x_s=x[n:int(i),:]
           x_sub.append(x_s)
           p_s=p[n:int(i),:]
           p_sub.append(p_s)
           tgt_len, embed_dim = x_s.size() ###(N,d) 

           n=int(i)

           q = self.q_proj(x_s)######
        # (N,d)
           k = self.k_proj(x_s)########
        # (N,d)
           v = self.v_proj(x_s)######

           pe=self.linear_p(p_s)
    
           q = q+pe
           k = k+pe
        # activation
           q = self.act_fun(q)####Q'
           k = self.act_fun(k)####K'
        
        # cos transform
           m = tgt_len
        # get index and send to cuda
           weight_index = self.get_index(m).to(q)########(pi*[1,m,1])/2

        # (N, 2 * d)
           q_ = torch.cat([q * torch.sin(weight_index[:tgt_len, :] / m), q * torch.cos(weight_index[ :tgt_len, :] / m)], dim=-1).squeeze(0)
        # (N, 2 * d)
           k_ = torch.cat([k * torch.sin(weight_index[:tgt_len, :] / m), k * torch.cos(weight_index[ :tgt_len, :] / m)], dim=-1).squeeze(0)

        
            # ( L, 2 * d) ( L, d) -> ( 2 * d, d)
           kv_ = torch.einsum('ld,lm->dm', k_, v)
            # ( L, 2 * d) ( 2 * d) -> ( L)
           z_ = 1 / torch.clamp_min(torch.einsum('ld,d->l', q_, torch.sum(k_, axis=0)), eps)
            # ( L, 2 * d) ( d, 2 * d) ( L) -> ( L, d)
           attn_output = self.out_proj(torch.einsum('ld,dm,l->lm', q_, kv_, z_))
           out.append(attn_output)
        #out=torch.IntTensor(out)
        out=torch.cat(out,dim=0)

        return out